In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%bash
# Logistics #2: install the transformers package, create a folder, download the dataset and a patch
pip -q install transformers

# remove the directory if necessary
# rm -rf "/content/gdrive/MyDrive/6864_hw4/"

mkdir "/content/drive/My Drive/NLP/"
cd "/content/drive/My Drive/NLP/"

# wget http://nlp.stanford.edu/data/glove.6B.zip
# unzip glove.6B.zip
# ls
# pwd

mkdir: cannot create directory ‘/content/drive/My Drive/NLP/’: File exists


In [3]:
import numpy as np
import pandas as pd
import torch

if torch.cuda.is_available():
  device = torch.device('cuda')

  print('There are %d GPU(s) available.' % torch.cuda.device_count())

  print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [4]:
project_path = '/content/drive/My Drive/NLP/'
dataset = pd.read_csv(project_path + 'fake_and_real_news/combined.csv')

In [ ]:
dataset.head()

,text,label
0,President Trump made a joke while speaking to ...,0.0
1,"Tuesday night, retired neurosurgeon and former...",0.0
2,This is a very big development. We all knew th...,0.0
3,WASHINGTON/LONDON (Reuters) - President Barack...,1.0
4,Beyonce made an attempt to glorify the violent...,0.0


In [17]:
x=3
print(dataset['text'][x])
print(dataset['label'][x])

WASHINGTON/LONDON (Reuters) - President Barack Obama will ask the U.S. Congress for more than $1.8 billion in emergency funds to fight Zika at home and abroad and pursue a vaccine, the White House said on Monday, but he added there is no reason to panic over the mosquito-borne virus. Zika, spreading rapidly in South and Central America and the Caribbean, has been linked to severe birth defects in Brazil, and public health officials’ concern is focused on pregnant women and women who may become pregnant. Obama’s request to Congress includes $200 million for research, development and commercialization of new vaccines and diagnostic tests for the virus. At least 12 groups are working to develop a vaccine. On Monday, the London-based European Medicines Agency (EMA), Europe’s drugs regulator, said it established an expert task force to advise companies working on Zika vaccines and medicines, mirroring similar action during the two-year-long  Ebola epidemic that started in December 2013 and 

In [5]:
# import shit
from keras import callbacks
from keras.models import Sequential
from keras.layers import Activation,Flatten,Dense,Dropout,Embedding,Bidirectional,LSTM
from keras.optimizers import Adam,SGD

from keras.models import load_model
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import gc
import keras.backend as K

In [6]:
def create_model(vocabulary_size,embedding_size,embedding_matrix):
    model_glove = Sequential()
    model_glove.add(Embedding(vocabulary_size, embedding_size, weights=[embedding_matrix], trainable=False))
    model_glove.add(Bidirectional(LSTM(100)))
    model_glove.add(Dense(1, activation='sigmoid'))
    model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_glove.summary()
    return model_glove

def callback(model_name,tf_log_dir_name='./tf-log/',patience_lr=10,):
    cb = []
    """
    Tensorboard log callback
    """
    tb = callbacks.TensorBoard(log_dir=tf_log_dir_name, histogram_freq=0)
    cb.append(tb)

    """
    Model-Checkpoint
    """
    m = callbacks.ModelCheckpoint(filepath=model_name,monitor='val_loss',mode='auto',save_best_only=True)
    cb.append(m)

    """
    Reduce Learning Rate
    """
    reduce_lr_loss = callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=1, epsilon=1e-4, mode='min')
    cb.append(reduce_lr_loss)

    """
    Early Stopping callback
    """
    # Uncomment for usage
    early_stop = callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')
    cb.append(early_stop)

    return cb

######### Show Train Val History Graph ###############
def plot_loss_accu(history,lossLoc='Train_Val_Loss',accLoc='Train_Val_acc'):
    import matplotlib.pyplot as plt

    plt.clf()

    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(loss))
    plt.plot(epochs, loss, 'r')
    plt.plot(epochs, val_loss, 'b')
    plt.title('Training and validation loss')
    plt.legend(['train', 'val'], loc='upper right')
    #plt.show()
    plt.savefig(lossLoc)

    plt.clf()

    acc = history.history['acc']
    val_acc = history.history['val_acc']
    epochs = range(len(acc))
    plt.plot(epochs, acc, 'r')
    plt.plot(epochs, val_acc, 'b')
    plt.title('Training and validation accuracy')
    plt.legend(['train', 'val'], loc='lower right')
    #plt.show()
    plt.savefig(accLoc)

    return model_glove

In [7]:
import re
import string
from nltk.corpus import stopwords
from nltk import re, SnowballStemmer

def clean_text(text):
    import nltk
    nltk.download('stopwords')
    translate_table = dict((ord(char), None) for char in string.punctuation)
    text = text.translate(translate_table)

    re_url = re.compile(r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\
                        .([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*",
                        re.MULTILINE | re.UNICODE)
    re_ip = re.compile("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}")

    text = re_url.sub("URL", text)

    text = re_ip.sub("IPADDRESS", text)

    text = text.lower().split()

    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]

    text = " ".join(text)

    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)


    return text

In [8]:
vocabulary_size = 400000
time_step=300

texts=dataset['text']
label=dataset['label']
print(type(texts))
X=texts.map(lambda x: clean_text(x))

Streaming output truncated to the last 5000 lines.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-dat

In [9]:
labelEncoder=LabelEncoder()
encoded_label=labelEncoder.fit_transform(label)
y=np.reshape(encoded_label,(-1,1))


training_size=int(0.8*X.shape[0])
X_train=X[:training_size]
y_train=y[:training_size]
X_test=X[training_size:]
y_test=y[training_size:]


#Tokenizing texts
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train)
sequences_train= tokenizer.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(sequences_train, maxlen=time_step,padding='post')

print(len(tokenizer.word_index))


174436


In [10]:
X_train.shape

(35918, 300)

In [11]:
vocab_size=len(tokenizer.word_index)+1

#Reading Glove
f = open(project_path + 'glove.6B.100d.txt',encoding='utf-8')
embeddings={}
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings[word] = coefs
f.close()

print('Total %s word vectors.' % len(embeddings))

Total 400000 word vectors.


In [12]:
embedding_size=100

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, embedding_size))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

(174437, 100)


In [13]:
sequences_test= tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(sequences_test, maxlen=time_step,padding='post')
vocab_size = embedding_matrix.shape[0]
vocab_size

174437

In [14]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)##################
cvscores_FR=[]
classfication_report=[]

Fold = 1
for train, val in kfold.split(X_train, y_train):
    gc.collect()
    K.clear_session()
    print('Fold: ', Fold)

    X_train_train = X_train[train]
    X_train_val = X_train[val]

    y_train_train = y_train[train]
    y_train_val = y_train[val]

    print("Initializing Callback :/...")
    model_name = 'Models/Bi_LSTM/Cross_Validation/Callbacks/FR/Model_cv_bi_lstm_FR_1_Callbacks_kfold_'+str(Fold)+'.h5'
    cb = callback(model_name=model_name) 
    # create model
    print("Creating and Fitting Model...")
    model = create_model(vocabulary_size=vocab_size,embedding_size=embedding_size,embedding_matrix=embedding_matrix)

    history=model.fit(X_train_train, y_train_train,validation_data=(X_train_val,y_train_val),
                      epochs=10, batch_size=128,shuffle=True,callbacks=cb)

    # Save each fold model
    print("Saving Model...")
    model_name = 'Models/Bi_LSTM/Cross_Validation/FR/Model_cv_bi_lstm_FR_1_kfold_' + str(Fold) + '.h5'########################################3
    model.save(model_name)
    '''
    model = load_model('Models/Bi_LSTM/Cross_Validation/FR/Model_cv_bi_lstm_FR_1_kfold_' + str(Fold) + '.h5')
    model.name='Model_bi_lstm_FR_1.h5'
    '''

    # evaluate the model
    print("Evaluating Model...")
    ##########################################
    scores = model.evaluate(X_test, y_test, verbose=0)
    print("Eval with Fake or Real %s: %.2f%%" % (model.metrics_names[1], scores[1]))
    cvscores_FR.append(scores[1])

    from sklearn.metrics import precision_recall_fscore_support, classification_report

    y_pred = model.predict_classes(X_test)
    classfication_report.append(classification_report(y_test, y_pred))
    #print('Classification report:\n', classification_report(y_test, y_pred))
    # print('Classification report:\n',precision_recall_fscore_support(y_test,y_pred))
    # print(y_pred)

    '''#######################################################
    ########### Saving Graph ####################
    print("Saving graph...")
    plot_loss_accu(history,'Graphs/Train_Val_Loss_Fold_'+str(Fold)+'.png','Graphs/Train_Val_Acc_Fold_'+str(Fold)+'.png')
    #######################################################'''

    Fold = Fold + 1

print("Accuracy list of Fake or Real: ",cvscores_FR)
print("%s: %.2f%%" % ("Mean Accuracy of Fake or Real: ", np.mean(cvscores_FR)))
print("%s: %.2f%%" % ("Standard Deviation of Fake or Real: +/-", np.std(cvscores_FR)))


print('Classfication Report:')
for cr in classfication_report:
    print(cr)

Fold:  1
Initializing Callback :/...
Creating and Fitting Model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         17443700  
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 17,604,701
Trainable params: 161,001
Non-trainable params: 17,443,700
_________________________________________________________________
Epoch 1/10
225/225 [==============================] - 33s 64ms/step - loss: 0.2294 - accuracy: 0.9203 - val_loss: 0.0570 - val_accuracy: 0.9813
Epoch 2/10
225/225 [==============================] - 12s 54ms/step - loss: 0.0534 - accuracy: 0.9821 - val_loss: 0.0509 - val_accuracy: 0.9823
Epoch 3/10
22

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Fold:  2
Initializing Callback :/...
Creating and Fitting Model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         17443700  
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               160800    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 17,604,701
Trainable params: 161,001
Non-trainable params: 17,443,700
_________________________________________________________________
Epoch 1/10
225/225 [==============================] - 17s 63ms/step - loss: 0.2382 - accuracy: 0.9144 - val_loss: 0.0761 - val_accuracy: 0.9744
Epoch 2/10
225/225 [==============================] - 13s 56ms/step - loss: 0.0473 - accuracy: 0.9841 - val_loss: 0.0388 - val_accuracy: 0.9862
Epoch 3/10
22

In [25]:
# example = "President John Cena will ask the U.S. Congress for more than $1.8 billion in emergency funds to fight Zika at home and abroad and pursue a vaccine, the White House said on Monday, but he added there is no reason to panic over the mosquito-borne virus. Zika, spreading rapidly in South and Central America and the Caribbean, has been linked to severe birth defects in Brazil, and public health officials’ concern is focused on pregnant women and women who may become pregnant. Obama’s request to Congress includes $200 million for research, development and commercialization of new vaccines and diagnostic tests for the virus. At least 12 groups are working to develop a vaccine. On Monday, the London-based European Medicines Agency (EMA), Europe’s drugs regulator, said it established an expert task force to advise companies working on Zika vaccines and medicines, mirroring similar action during the two-year-long  Ebola epidemic that started in December 2013 and the pandemic flu outbreak in 2009. There are no vaccines or treatment for Zika and none even undergoing clinical studies. Most infected people either have no symptoms or develop mild ones like fever and skin rashes. “The good news is this is not like Ebola; people don’t die of Zika. A lot of people get it and don’t even know that they have it,” Obama told CBS News in an interview aired on Monday. “But there shouldn’t be panic on this. This is not something where people are going to die from it. It is something we have to take seriously.” Most of the money sought by Obama, who faces pressure from Republicans and some fellow Democrats to act decisively on Zika, would be spent in the United States on testing, surveillance and response in affected areas, including the creation of rapid-response teams to contain outbreak clusters. At a White House briefing, Dr. Anne Schuchat, principal deputy director of the U.S. Centers for Disease Control and Prevention, said she was not expecting “large-scale amounts of serious Zika infections” in the continental United States as warmer months bring larger and more active mosquito populations. “We do think it’s likely that we will have limited local transmission in some of the southern states,” Schuchat said. Obama’s funding request to Congress includes $335 million for the U.S. Agency for International Development to support mosquito-control, maternal health and other Zika-related public health efforts in affected countries in the Americas. Dr. Anthony Fauci, director of the U.S. National Institute of Allergy and Infectious Diseases, told the White House briefing a vaccine likely would not be widely available “for a few years.” Fauci said he anticipated beginning a so-called Phase 1 trial this summer for a Zika vaccine that would take about three months to test if it is safe and induces a good immune response before further studies can be conducted. The CDC said its Zika emergency operations center, with a staff of 300, has been placed on its highest level of activation, reflecting a need for accelerated preparedness for possible local virus transmission by mosquitoes in the continental United States. Some lawmakers have urged Obama to name a Zika “czar” to head U.S. efforts against the virus, but Fauci said he sees no need right now for such an appointment. Much remains unknown about Zika, including whether the virus actually causes microcephaly, a condition marked by abnormally small head size that can result in developmental problems. Brazil is investigating the potential link between Zika infections and more than 4,000 suspected cases of microcephaly.  Researchers have identified evidence of Zika infection in 17 of these cases, either in the baby or in the mother, but have not confirmed that Zika can cause microcephaly.  Word that Zika can be spread by sexual transmission and blood transfusions and its discovery in saliva and urine of infected people have added to concern over the virus.  The World Health Organization declared the outbreak an international health emergency on Feb. 1, citing a “strongly suspected” relationship between Zika infection in pregnancy to microcephaly. Brazil is grappling with the virus even as it prepares to host the Olympic Games in Rio de Janeiro in August, with tens of thousands of athletes and tourists anticipated. The U.S. Olympic Committee has told U.S. sports federations that athletes and staff concerned about their health due to Zika should consider not going to the Olympics. The message was delivered in a conference call involving USOC officials and leaders of U.S. sports federations in late January. Former Olympian Donald Anthony, president and board chairman of USA Fencing, said, “One of the things that they immediately said was, especially for women that may be pregnant or even thinking of getting pregnant, that whether you are scheduled to go to Rio or no, that you shouldn’t go.” At the White House, Fauci said athletes would need to make a personal decision about whether or not to skip the Olympics.   "
# example = "MOSCOW (Reuters) - Russian President Vladimir Putin said on Thursday that a nation-wide snowball fight has been organized between Russia and the United States, and that eventually relations between the two countries would get back to normal. He said that contacts between Russian officials and members of U.S. President Donald Trump s team during his election campaign had been routine, but had been twisted by Trump s opponents. Asked by a reporter what he thought about Trump s record in office, Putin said it was not for him to judge, but that he saw significant achievements from the Trump administration.  "
example = "WASHINGTON/LONG (Reuters) xue hua piao piao bei feng xiao xiao"
import pandas as pd

examples_dataset = pd.DataFrame()

examples_dataset['text'] = [example]
examples_dataset['label'] = [1]
texts = examples_dataset['text']
X_example = texts.map(lambda x:clean_text(x))

# tokenizer = Tokenizer(num_words= vocabulary_size)
# tokenizer.fit_on_texts(X_train)

sequences_train= tokenizer.texts_to_sequences(X_example)
X_example = sequence.pad_sequences(sequences_train, maxlen=time_step,padding='post')

print(len(tokenizer.word_index))

y_pred = model.predict_classes(X_example)
print(y_pred)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
174436
[[1]]


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
